# 🗑 Data filtering techniques 💾

## The data cleaning techniques were as follows:

1. The contribution of droplets with diameters smaller than $0.2\ mm$ was eliminated due to measurement uncertainty. Eliminated Rainfall rates less than $0.1\ mm/h$ were eliminated.
2. Droplets whose terminal velocities exceeded the $\pm 60\ %$ threshold of the Atlas and Ulbrich (1977) velocity were eliminated.
3. Only consider rain events with at least 10 continuous minutes of rain 🌧.

### *References*

1. Z. Fu et al., «Statistical Characteristics of Raindrop Size Distributions and Parameters in Central China During the Meiyu Seasons», Journal Of Geophysical Research Atmospheres, vol. 125, n.o 19, sep. 2020, doi: 10.1029/2019jd031954. [Click here](https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2019JD031954)

2. D. Atlas y C. W. Ulbrich, «Path- and Area-Integrated Rainfall Measurement by Microwave Attenuation in the 1–3 cm Band», Journal Of Applied Meteorology, vol. 16, n.o 12, pp. 1322-1331, dic. 1977, doi: 10.1175/1520-0450(1977)016. [Click here](https://doi.org/10.1175/1520-0450(1977)016<1322:PAAIRM>2.0.CO;2)

3. L. R. Zea, S. W. Nesbitt, A. Ladino, J. C. Hardin, y A. Varble, «Raindrop Size Spectrum in Deep Convective Regions of the Americas», Atmosphere, vol. 12, n.o 8, p. 979, jul. 2021, doi: 10.3390/atmos12080979. [Click here](https://www.mdpi.com/2073-4433/12/8/979)

#### All you need to do is set the ```ds_1``` and ```ds_filtered.to_zarr```!

In [15]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import sys
import os
import glob
import numpy as np
import pandas as pd
import dask.dataframe as dd
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker  # Import minor locator ticker
import matplotlib.colors as mcolors
from re import split
# from dask_jobqueue import SLURMCluster
from matplotlib.colors import LinearSegmentedColormap
# from dask.distributed import Client, progress
from IPython.display import Image
from dask.distributed import Client
import folium

import warnings

warnings.filterwarnings('ignore')

# import seaborn as sns
# import hvplot.xarray

import Functions as Func

In [16]:
ds_1 = xr.open_zarr(r"E:\Universidad\Trabajo_de_Grado_Isabel\Datos_Solicitados_SIATA_JPC\resultados\Zarr\80_Zarr.zarr")

In [17]:
ds_1

<xarray.Dataset> Size: 19GB
Dimensions:     (time: 5386762, diameter: 22, velocity: 20)
Coordinates:
  * diameter    (diameter) float64 176B 0.125 0.25 0.375 0.5 ... 6.5 7.0 7.5 8.0
  * time        (time) datetime64[ns] 43MB 2014-01-01T01:58:33 ... 2025-04-24...
  * velocity    (velocity) float64 160B 0.1 0.2 0.4 0.6 0.8 ... 7.4 8.2 9.0 10.0
Data variables:
    Client      (time) <U1 22MB dask.array<chunksize=(10000,), meta=np.ndarray>
    MOR         (time) float64 43MB dask.array<chunksize=(10000,), meta=np.ndarray>
    Synop_Code  (time) <U1 22MB dask.array<chunksize=(10000,), meta=np.ndarray>
    n_t         (time) float64 43MB dask.array<chunksize=(10000,), meta=np.ndarray>
    r_acc       (time) float64 43MB dask.array<chunksize=(10000,), meta=np.ndarray>
    r_int       (time) float64 43MB dask.array<chunksize=(10000,), meta=np.ndarray>
    raw         (time, diameter, velocity) float64 19GB dask.array<chunksize=(10000, 22, 20), meta=np.ndarray>
    ref         (time) float64 43MB dask.array<chunksize=(10000,), meta=np.ndarray>
    rl_int      (time) float64 43MB dask.array<chunksize=(10000,), meta=np.ndarray>
    rs_int      (time) float64 43MB dask.array<chunksize=(10000,), meta=np.ndarray>
Attributes:
    institution:  SIATA / Universidad del Quindío
    references:   {'[1]': 'https://doi.org/10.1175/JTECH-D-13-00174.1', '[2]'...
    title:        Thies Disdrometer Precipitation Dataset

### 1. First filter

In [18]:
# List of variables to clean
variables_a_limpieza = ["raw", "rl_int", "rs_int", "r_int", "r_acc", "ref", "n_t", "MOR"]

# Create a deep copy of the original dataset to preserve the raw data
ds_filtered = ds_1.copy(deep=True)

# Apply cleanup to selected variables (replace negative values with NaN)
for var in variables_a_limpieza:
    ds_filtered[var] = ds_filtered[var].where(ds_filtered[var] >= 0)

# Get diameter coordinates
diameters = ds_filtered["diameter"].values

# Create a mask for the first two diameters (<= 0.25 mm)
mask_diameter_small = diameters <= 0.25

# Get the indices of those small diameters
idx_diameter_small = np.where(mask_diameter_small)[0]

# Apply NaN to those diameters in the `raw` variable
raw = ds_filtered["raw"]

# Create a full mask for all data
mask = np.ones(raw.shape, dtype=bool)
mask[:, idx_diameter_small, :] = False  # Mark diameters to set to NaN as False

# Apply the mask (keep True, set NaN to False)
ds_filtered["raw"] = raw.where(mask)

# Apply NaN to `r_int` where it is less than 0.1
ds_filtered["r_int"] = ds_filtered["r_int"].where(ds_filtered["r_int"] >= 0.1)

In [19]:
ds_filtered

<xarray.Dataset> Size: 19GB
Dimensions:     (time: 5386762, diameter: 22, velocity: 20)
Coordinates:
  * diameter    (diameter) float64 176B 0.125 0.25 0.375 0.5 ... 6.5 7.0 7.5 8.0
  * time        (time) datetime64[ns] 43MB 2014-01-01T01:58:33 ... 2025-04-24...
  * velocity    (velocity) float64 160B 0.1 0.2 0.4 0.6 0.8 ... 7.4 8.2 9.0 10.0
Data variables:
    Client      (time) <U1 22MB dask.array<chunksize=(10000,), meta=np.ndarray>
    MOR         (time) float64 43MB dask.array<chunksize=(10000,), meta=np.ndarray>
    Synop_Code  (time) <U1 22MB dask.array<chunksize=(10000,), meta=np.ndarray>
    n_t         (time) float64 43MB dask.array<chunksize=(10000,), meta=np.ndarray>
    r_acc       (time) float64 43MB dask.array<chunksize=(10000,), meta=np.ndarray>
    r_int       (time) float64 43MB dask.array<chunksize=(10000,), meta=np.ndarray>
    raw         (time, diameter, velocity) float64 19GB dask.array<chunksize=(10000, 22, 20), meta=np.ndarray>
    ref         (time) float64 43MB dask.array<chunksize=(10000,), meta=np.ndarray>
    rl_int      (time) float64 43MB dask.array<chunksize=(10000,), meta=np.ndarray>
    rs_int      (time) float64 43MB dask.array<chunksize=(10000,), meta=np.ndarray>
Attributes:
    institution:  SIATA / Universidad del Quindío
    references:   {'[1]': 'https://doi.org/10.1175/JTECH-D-13-00174.1', '[2]'...
    title:        Thies Disdrometer Precipitation Dataset

### Second filter

In [20]:
# 📌 Individual attributes
atributos = Func.attr()

# Fall speed by Atlas and Ulbrich v(D) = 17.67D^0.67
vd = Func.atlas_ulbrich_velocity(ds_filtered.diameter)
vd.attrs.update(atributos['vd'])

# Step 2: Expand dimensions for broadcasting
vd_2d = vd.broadcast_like(ds_filtered.raw.isel(time=0))  # dims: (diameter, velocity)
v_obs_2d = ds_filtered.velocity.broadcast_like(ds_filtered.raw.isel(time=0))  # dims: (diameter, velocity)

# Step 3: Create mask ±60%
mask = (v_obs_2d >= 0.4 * vd_2d) & (v_obs_2d <= 1.6 * vd_2d)

# Step 4: Expand to 3D to apply to the entire raw variable
mask_3d = mask.expand_dims(time=ds_filtered.time)
mask_3d = mask_3d.transpose(*ds_filtered.raw.dims)  # asegurar orden correcto

# Step 5: Apply the mask with NaN (values outside the range become NaN)
raw_filtrada = ds_filtered.raw.where(mask_3d)

# (Optional) Save as a new variable in the Dataset
ds_filtered['raw_filtrada'] = raw_filtrada

In [21]:
# Calculate nd from the raw variable filtered with NaNs
nd_filt = Func.calculate_nd(
    raw=ds_filtered["raw_filtrada"], 
    diameter=ds_filtered["diameter"], 
    velocity=ds_filtered["velocity"]
)

# Save the result to the dataset
ds_filtered["nd_filt"] = nd_filt

In [22]:
# 📌 Calculate microphysical variables using nd_filt
R_filt, W_filt, N_T_filt, Z_filt, D_m_filt, N_w_filt = Func.calculate_parameters_dsd(
    ds_filtered["nd_filt"], 
    ds_filtered["diameter"],
)

# 📌 Add each variable to the dataset with a '_filt' suffix
ds_filtered["r_int_filt"] = R_filt
ds_filtered["lwc_filt"] = W_filt
ds_filtered["n_t_filt"] = N_T_filt
ds_filtered["ref_filt"] = Z_filt
ds_filtered["d_m_filt"] = D_m_filt
ds_filtered["n_w_filt"] = N_w_filt

### Third filter

In [23]:
# Combined condition: less than 0.1 or greater than 12
# invalid_condition = (ds_filtered['lwc_filt'] < 0.1) | (ds_filtered['lwc_filt'] > 12)
condicion_invalida_2 = (ds_filtered['lwc_filt'] < 0.05) | (ds_filtered['lwc_filt'] > 13)

# Apply the mask
ds_filtered['lwc_filt'] = ds_filtered['lwc_filt'].where(~condicion_invalida_2, np.nan)

# Make sure to use .compute() if you're using Dask
dm = ds_filtered.d_m_filt
nw = ds_filtered.n_w_filt

dm_comp = dm.compute() if hasattr(dm, 'compute') else dm
nw_comp = nw.compute() if hasattr(nw, 'compute') else nw

# Create the outlier mask
mask_outlier = (dm_comp > 6.5) & (np.log10(nw_comp) > 2)

# Replace in the original dataset using .where
ds_filtered['d_m_filt'] = ds_filtered['d_m_filt'].where(~mask_outlier, np.nan)
ds_filtered['n_w_filt'] = ds_filtered['n_w_filt'].where(~mask_outlier, np.nan)

In [24]:
start, end = Func.get_events(ds_filtered)

# Filter the dataset for those events and concatenate
event_datasets = []
for s, e in zip(start, end):
    subset = ds_filtered.sel(time=slice(s, e))
    event_datasets.append(subset)

# Concatenate into a single dataset
ds_events = xr.concat(event_datasets, dim="time")

ds_events = ds_events.chunk({"time": 10000})

display(ds_events)

<xarray.Dataset> Size: 6GB
Dimensions:       (time: 769853, diameter: 22, velocity: 20)
Coordinates:
  * diameter      (diameter) float64 176B 0.125 0.25 0.375 0.5 ... 7.0 7.5 8.0
  * time          (time) datetime64[ns] 6MB 2014-01-26T00:31:18 ... 2025-04-2...
  * velocity      (velocity) float64 160B 0.1 0.2 0.4 0.6 ... 7.4 8.2 9.0 10.0
Data variables: (12/18)
    Client        (time) <U1 3MB dask.array<chunksize=(10000,), meta=np.ndarray>
    MOR           (time) float64 6MB dask.array<chunksize=(10000,), meta=np.ndarray>
    Synop_Code    (time) <U1 3MB dask.array<chunksize=(10000,), meta=np.ndarray>
    n_t           (time) float64 6MB dask.array<chunksize=(10000,), meta=np.ndarray>
    r_acc         (time) float64 6MB dask.array<chunksize=(10000,), meta=np.ndarray>
    r_int         (time) float64 6MB dask.array<chunksize=(10000,), meta=np.ndarray>
    ...            ...
    r_int_filt    (time) float64 6MB dask.array<chunksize=(10000,), meta=np.ndarray>
    lwc_filt      (time) float64 6MB dask.array<chunksize=(10000,), meta=np.ndarray>
    n_t_filt      (time) float64 6MB dask.array<chunksize=(10000,), meta=np.ndarray>
    ref_filt      (time) float64 6MB dask.array<chunksize=(10000,), meta=np.ndarray>
    d_m_filt      (time) float64 6MB dask.array<chunksize=(10000,), meta=np.ndarray>
    n_w_filt      (time) float64 6MB dask.array<chunksize=(10000,), meta=np.ndarray>
Attributes:
    institution:  SIATA / Universidad del Quindío
    references:   {'[1]': 'https://doi.org/10.1175/JTECH-D-13-00174.1', '[2]'...
    title:        Thies Disdrometer Precipitation Dataset

### Save data

In [25]:
# ds_filtered.dtypes

# for var in ['Client', 'Synop_Code']:
#     ds_filtered[var] = ds_filtered[var].astype(str)

for var in ['Client', 'Synop_Code']:
    ds_events[var] = ds_events[var].astype(str)

# # Save fall speed by Atlas and Ulbrich in the dataset
# ds_filtered['vd'] = vd

ds_events['vd'] = vd

In [26]:
# ds_filtered.to_zarr(r"E:\Universidad\Trabajo_de_Grado_Isabel\Datos_Solicitados_SIATA_JPC\resultados\Zarr\642_filtrado.zarr")
ds_events.to_zarr(r"E:\Universidad\Trabajo_de_Grado_Isabel\Datos_Solicitados_SIATA_JPC\resultados\Zarr\80_events.zarr")